In [1]:
#| label: libraries
#| include: false
import pandas as pd
import altair as alt
import numpy as np
import seaborn as sns
import pandas as pd
import altair as alt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from prettytable import PrettyTable

## Elevator pitch

In this project, I tackled the Star Wars survey dataset, transforming it into a clean and structured format suitable for machine learning. I began by shortening and cleaning up column names for ease of use in pandas. Subsequently, I filtered the dataset to include only respondents who had seen at least one Star Wars film. Age ranges, education groupings, and income ranges were converted to single numerical representations, and respective categorical columns were dropped. The target column, indicating whether a person makes more than $50k, was created.

Following this, I one-hot encoded remaining categorical columns and validated the dataset's consistency with the provided article by recreating visuals. A machine learning model, employing a Decision Tree classifier, was built to predict income levels. The model achieved an accuracy of 59%, and further exploration involved identifying and utilizing important features to enhance performance. The resultant model with important features yielded a similar accuracy, showcasing the significance of feature selection.

This project demonstrates the process of data cleaning, preprocessing, and building a predictive model, contributing valuable insights into the Star Wars survey dataset.


In [2]:
#| label: project data
#| code-summary: Read and format project data
# Include and execute your code here
df = pd.read_csv(
    "https://raw.githubusercontent.com/fivethirtyeight/data/master/star-wars-survey/StarWars.csv",
    encoding="latin1",
)

__Highlight the grand questions__

## GRAND QUESTION 1

Shorten the column names and clean them up for easier use with pandas. Provide a table or list that exemplifies how you fixed the names.

I went into the data frame and looked at all of the column names, with the end goal in mind in order to change the column names to something that may be useful in creating the machine learning model. Using .rename, I shortened the names, took out any spaces, and made them all lowercase so that they would all have similar format. I then created a table using PrettyTable to showcase these new names.


In [3]:
#| label: GQ1
#| code-summary: Read and format data
# Include and execute your code here
df = df.rename(
    columns={
        "RespondentID": "id",
        "Have you seen any of the 6 films in the Star Wars franchise?": "anyfilm",
        "Do you consider yourself to be a fan of the Star Wars film franchise?": "fan",
        "Which of the following Star Wars films have you seen? Please select all that apply.": "seen1",
        "Unnamed: 4": "seen2",
        "Unnamed: 5": "seen3",
        "Unnamed: 6": "seen4",
        "Unnamed: 7": "seen5",
        "Unnamed: 8": "seen6",
        "Please rank the Star Wars films in order of preference with 1 being your favorite film in the franchise and 6 being your least favorite film.": "rank1",
        "Unnamed: 10": "rank2",
        "Unnamed: 11": "rank3",
        "Unnamed: 12": "rank4",
        "Unnamed: 13": "rank5",
        "Unnamed: 14": "rank6",
        "Please state whether you view the following characters favorably, unfavorably, or are unfamiliar with him/her.": "hansolo",
        "Unnamed: 16": "lukeskywalker",
        "Unnamed: 17": "leia",
        "Unnamed: 18": "anakin",
        "Unnamed: 19": "obiwan",
        "Unnamed: 20": "palpatine",
        "Unnamed: 21": "vader",
        "Unnamed: 22": "lando",
        "Unnamed: 23": "boba",
        "Unnamed: 24": "c3p0",
        "Unnamed: 25": "r2d2",
        "Unnamed: 26": "jarjar",
        "Unnamed: 27": "padme",
        "Unnamed: 28": "yoda",
        "Which character shot first?": "shotfirst",
        "Are you familiar with the Expanded Universe?": "familiarexpu",
        "Do you consider yourself to be a fan of the Expanded Universe?æ": "fanexpu",
        "Do you consider yourself to be a fan of the Star Trek franchise?": "fanstartrek",
        "Gender": "gender",
        "Age": "age",
        "Household Income": "income",
        "Education": "education",
        "Location (Census Region)": "location",
    }
)
df = df.drop_duplicates()

In [4]:
#| label: GQ1 chart
#| code-summary: plot example
#| fig-cap: New Column Name Table
#| fig-align: center
# Include and execute your code here
table = PrettyTable()
table.field_names = df.columns
print(table)

+----+---------+-----+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+---------+---------------+------+--------+--------+-----------+-------+-------+------+------+------+--------+-------+------+-----------+--------------+---------+-------------+--------+-----+--------+-----------+----------+
| id | anyfilm | fan | seen1 | seen2 | seen3 | seen4 | seen5 | seen6 | rank1 | rank2 | rank3 | rank4 | rank5 | rank6 | hansolo | lukeskywalker | leia | anakin | obiwan | palpatine | vader | lando | boba | c3p0 | r2d2 | jarjar | padme | yoda | shotfirst | familiarexpu | fanexpu | fanstartrek | gender | age | income | education | location |
+----+---------+-----+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+---------+---------------+------+--------+--------+-----------+-------+-------+------+------+------+--------+-------+------+-----------+--------------+---------+-------------+--------+-----+--------+-

## GRAND QUESTION 2

Clean and format the data so that it can be used in a machine learning model. As you format the data, you should complete each item listed below. In your final report provide example(s) of the reformatted data with a short description of the changes made.
a. Filter the dataset to respondents that have seen at least one film.
b. Create a new column that converts the age ranges to a single number. Drop the age range categorical column.
c. Create a new column that converts the education groupings to a single number. Drop the school categorical column
d. Create a new column that converts the income ranges to a single number. Drop the income range categorical column.
e. Create your target (also known as “y” or “label”) column based on the new income range column.
One-hot encode all remaining categorical columns.

For part a, I filtered the df to make sure that only people who had actually seen at least one film were on it. I originally just used the answers from the "Have you seen any of the 6 films in the Star Wars franchise?" column, but later on found discrepencies in the data and realized that some people had said yes, but then had not actually said they had seen a single one of the movies asked about. Because of this, I added a way to check if they had said yes to seeing any one of the 6 movies and said yes to seeing one.

For part b I calculated midpoint ages using a function, and rather than creating a brand new column and then deleting the old one, I just replaced the old column with this new one.

For part c I replaced all of the education groupings with numbers l-5, starting at "Less than high school degree" and ending with "Graduate degree".

For part d I followed the exact same protocol as part b, but for the income column.

For part e I took the midpoints and I made them into a single number encoding (1-5), from there I took anything over a 2 (in other words, over $50,000) and I turned those into a 1, and anything 2 or below I turned into a 0. This way it was clear to see, 0 is under 50k a year and 1 is over.

For part f I replaced all other strings with numbers. If they were binary questions, I used 1s and 0s, for other such as location I used 1-9 etc. Anything that was "unfamiliar" or "I don't understand" I also used 0.

In [5]:
#| label: GQ2
#| code-summary: Read and format data
# Include and execute your code here
df = df[
    (df["anyfilm"] == "Yes")
    & (
        (df["seen1"] == "Star Wars: Episode I  The Phantom Menace")
        | (df["seen2"] == "Star Wars: Episode II  Attack of the Clones")
        | (df["seen3"] == "Star Wars: Episode III  Revenge of the Sith")
        | (df["seen4"] == "Star Wars: Episode IV  A New Hope")
        | (df["seen5"] == "Star Wars: Episode V The Empire Strikes Back")
        | (df["seen6"] == "Star Wars: Episode VI Return of the Jedi")
    )
]

def calculate_midpoint(age):
    if pd.notna(age):
        if "-" in age:
            start, end = map(int, age.split("-"))
            return (start + end) / 2
        elif ">" in age:
            return int(age.split(">")[1])
        else:
            return int(age)
    else:
        return np.nan


df["age"] = df["age"].apply(calculate_midpoint)

def calculate_midpoint_income(income):
    if pd.notna(income):
        income_numeric = income.replace("$", "").replace(",", "").replace("+", "")

        if "-" in income_numeric:
            start, end = map(int, income_numeric.split("-"))
            return (start + end) / 2
        elif ">" in income_numeric:
            return int(income_numeric.split(">")[1])
        else:
            return int(income_numeric)
    else:
        return np.nan

df["income"] = df["income"].apply(calculate_midpoint_income)

df = df.replace(
    {
        "Star Wars: Episode I  The Phantom Menace": 1,
        "Star Wars: Episode II  Attack of the Clones": 1,
        "Star Wars: Episode III  Revenge of the Sith": 1,
        "Star Wars: Episode IV  A New Hope": 1,
        "Star Wars: Episode V The Empire Strikes Back": 1,
        "Star Wars: Episode VI Return of the Jedi": 1,
        "Yes": 1,
        "No": 0,
        "Unfamiliar (N/A)": 0,
        "Very unfavorably": 1,
        "Somewhat unfavorably": 2,
        "Neither favorably nor unfavorably (neutral)": 3,
        "Somewhat favorably": 4,
        "Very favorably": 5,
        "Female": 0,
        "Male": 1,
        "I don't understand this question": 0,
        "Greedo": 1,
        "Han": 2,
        "Less than high school degree": 1,
        "High school degree": 2,
        "Some college or Associate degree": 3,
        "Bachelor degree": 4,
        "Graduate degree": 5,
        "South Atlantic": 1,
        "West South Central": 2,
        "West North Central": 3,
        "Middle Atlantic": 4,
        "East North Central": 5,
        "Pacific": 6,
        "Mountain": 7,
        "New England": 8,
        "East South Central": 9,
        23.5: 1,
        37.0: 2,
        52.5: 3,
        60.0: 4,
        12499.5: 1,
        37499.5: 2,
        74999.5: 3,
        124999.5: 4,
        150000.0: 5
    }
)

df['target'] = df['income'].replace({1: 0, 2: 0, 3: 1, 4: 1, 5: 1})

## GRAND QUESTION 3

Validate that the data provided on GitHub lines up with the article by recreating 2 of the visuals from the article.

I chose to duplicate the "Which 'Star Wars' Movies Have You Seen?" and the "Who Shot First" charts from the article. I had an initial issue because some people had said they had seen a movie without actually recording which movie they had seen, so I had to take some time to go back and fix that (see GQ2 pt a). In the end I was able to make these charts almost identical in data and formatting. My data for the percentage of people who had seen "The Phantom Menace" did end up being off by 1%, but after double and triple checking, it looked like the raw data point would have been ~80.6%, so my rounding to 81% is correct. It took a bit of work to get the percentages to show on the right side of the bars, but I was able to get it done on both charts, just to make them look more identical to the article. 


In [6]:
#| label: GQ3
#| code-summary: Read and format data
# Include and execute your code here
alt.data_transformers.enable("default", max_rows=None)

movies_seen_df = df[["seen1", "seen2", "seen3", "seen4", "seen5", "seen6"]]
movies_seen_df = movies_seen_df.rename(
    columns={
        "seen1": "The Phantom Menace",
        "seen2": "Attack of the Clones",
        "seen3": "Revenge of the Sith",
        "seen4": "A New Hope",
        "seen5": "The Empire Strikes Back",
        "seen6": "Return of the Jedi",
    }
)

df_melted = movies_seen_df.melt()
df_melted = df_melted[df_melted["value"] == 1]
total_viewers = df["anyfilm"].value_counts()

df_melted["percentage"] = (df_melted.groupby("variable")["value"].transform("sum") / total_viewers[1] * 100).fillna(0).round(0)

df_melted["percentage_str"] = df_melted["percentage"].astype(int).astype(str) + "%"

In [7]:
#| label: GQ3 chart
#| code-summary: plot example
#| fig-cap: Which 'Star Wars' Movies Have You Seen?
#| fig-align: center
# Include and execute your code here
chart = (
    alt.Chart(df_melted)
    .mark_bar()
    .encode(
        y=alt.Y(
            "variable:N",
            title="",
            sort=[
                "The Phantom Menace",
                "Attack of the Clones",
                "Revenge of the Sith",
                "A New Hope",
                "The Empire Strikes Back",
                "Return of the Jedi",
            ],
        ),
        x=alt.X("sum(value):Q", title="", axis=None),
        tooltip=["variable:N", alt.Tooltip("percentage_str:N", title="Percentage")],
    )
    .properties(title="Which 'Star Wars' Movies Have You Seen?")
)

chart_text = chart.mark_text(
    align="left", baseline="middle", dx=3
).encode(text=alt.Text("percentage_str:N"))

chart_final = (
    chart + chart_text
).properties(title="Which 'Star Wars' Movies Have You Seen?")

chart_final

C:\Users\joshu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\altair\utils\core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\joshu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\altair\utils\core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

In [8]:
#| label: GQ3 pt 2
#| code-summary: Read and format data
# Include and execute your code here
df_filtered_shotfirst = df[df["shotfirst"] != ""]
df_filtered_shotfirst = df_filtered_shotfirst.replace(
    {0: "I don't understand this question", 1: "Greedo", 2: "Han"}
)

percentage_data = (
    df_filtered_shotfirst["shotfirst"].value_counts(normalize=True).reset_index()
)
percentage_data.columns = ["shotfirst", "percentage"]
percentage_data["percentage"] = percentage_data["percentage"].round(
    2
)

In [9]:
#| label: GQ3 chart 2
#| code-summary: table example
#| tbl-cap: Not much of a table
#| tbl-cap-location: top
# Include and execute your code here
chart_shotfirst_percentage = (
    alt.Chart(percentage_data)
    .mark_bar()
    .encode(
        y=alt.Y(
            "shotfirst:N",
            title="",
            sort=["Han", "Greedo", "I don't understand this question"],
        ),
        x=alt.X("percentage:Q", title="", axis=None),
        tooltip=[
            "shotfirst:N",
            alt.Tooltip("percentage:Q", title="Percentage", format=".0%"),
        ],
    )
)

chart_shotfirst_percentage_text = chart_shotfirst_percentage.mark_text(
    align="left", baseline="middle", dx=3
).encode(text=alt.Text("percentage:Q", format=".0%"))

chart_shotfirst_percentage_final = (
    chart_shotfirst_percentage + chart_shotfirst_percentage_text
).properties(title="Who Shot First?")

chart_shotfirst_percentage_final

C:\Users\joshu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\altair\utils\core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

__Highlight the grand questions__

## GRAND QUESTION 4

Build a machine learning model that predicts whether a person makes more than $50k. Describe your model and report the accuracy.

Here I created my machine learning model. I used all of the columns besides 'income' and 'target' in my X. I'm not sure if something went wrong, or if this data is just not great for making predictions, but the highest accuracy I could get was a 59%. I got the 59% while using all of the features, and then I even made a feature importance threshold in order to only include ones with a certain level of importance, but I still got a 59% when I ran that model with a .005 threshold. The only times that I was able to get a higher percentage were with a .015 threshold (63%) and a .02 threshold (67%), but those only had respectively 'vader' and 'age', or just 'age'. This shows that age is apparently a very large factor in how much money someone makes. Apparently your feelings on Darth Vader also play a big role here.

In [10]:
#| label: GQ4
#| code-summary: Read and format data
# Include and execute your code here
df['target'] = df['income'].replace({1: 0, 2: 0, 3: 1, 4: 1, 5: 1})
df = df.dropna(subset=['income'])
X = df.drop(columns = {'income', 'target'})
y = df['target']
X = pd.get_dummies(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)
y_pred = dt_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Decision Tree Accuracy: {accuracy:.2f}")
class_report = classification_report(y_test, y_pred)
print("Classification Report:\n", class_report)

Decision Tree Accuracy: 0.59
Classification Report:
               precision    recall  f1-score   support

         0.0       0.50      0.24      0.32        55
         1.0       0.61      0.84      0.71        80

    accuracy                           0.59       135
   macro avg       0.56      0.54      0.51       135
weighted avg       0.57      0.59      0.55       135



In [11]:
#| label: GQ4 chart
#| code-summary: plot example
#| fig-cap: Importance Chart
#| fig-align: center
# Include and execute your code here
dt_model.fit(X_train, y_train)
feature_importances = dt_model.feature_importances_
feature_names = X_train.columns
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
alt.Chart(feature_importance_df).mark_bar().encode(
    x=alt.X('Importance:Q', title='Importance'),
    y=alt.Y('Feature:N', title='Feature'),
).properties(title='Feature Importance Chart')

C:\Users\joshu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\altair\utils\core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [12]:
#| label: GQ4 2
#| code-summary: Read and format data
# Include and execute your code here
importance_threshold = 0.02
important_features = feature_importance_df[feature_importance_df['Importance'] > importance_threshold]
print("Important Features:")
print(important_features['Feature'].tolist())
selected_features = important_features['Feature'].tolist()
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]
dt_model_selected = DecisionTreeClassifier(random_state=42)
dt_model_selected.fit(X_train_selected, y_train)
y_pred_selected = dt_model_selected.predict(X_test_selected)
accuracy_selected = accuracy_score(y_test, y_pred_selected)
print(f"Decision Tree Accuracy with Important Features: {accuracy_selected:.2f}")
class_report_selected = classification_report(y_test, y_pred_selected)
print("Classification Report with Important Features:\n", class_report_selected)

Important Features:
['age']
Decision Tree Accuracy with Important Features: 0.67
Classification Report with Important Features:
               precision    recall  f1-score   support

         0.0       0.64      0.45      0.53        55
         1.0       0.69      0.82      0.75        80

    accuracy                           0.67       135
   macro avg       0.66      0.64      0.64       135
weighted avg       0.67      0.67      0.66       135



## APPENDIX A (Additional Python Code)

```python
#paste other your code from your python file (.py) here
```